<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [77]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

COMMON_PATH = '../../common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.driver.memory", "32g") \
    .config("spark.executor.memory", "32g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [78]:
DATA_PATH = '/home/stanislavv/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')
TEST_PATH = os.path.join(DATA_PATH, 'test.csv')
SUBMISSION_PATH = os.path.join(DATA_PATH, 'submission.csv')
DATA_PROCESSING_MODEL_CHECKPOINT = os.path.join(DATA_PATH, 'data_processing_model.bin')

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [4]:
df = df.sample(False, 0.5)

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [9]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(DATA_PROCESSING_MODEL_CHECKPOINT)

In [10]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

1833186

In [11]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

In [12]:
...

Ellipsis

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [13]:
!pip install hyperopt

     |████████████████████████████████| 1.9 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 4.1 MB/s eta 0:00:01
  Created wheel for networkx: filename=networkx-2.2-py2.py3-none-any.whl size=1526315 sha256=c72380cfad3f49c381edc4fcbc5e762a04adb22ddc37d593081e8dd6ce32afe7
  Stored in directory: /home/stanislavv/.cache/pip/wheels/d7/04/13/c8e685a798c75f7aa1f6b0a4aaf201bd6422bb933981cd47f7
Successfully built networkx
  Attempting uninstall: networkx
    Found existing installation: networkx 2.4
    Uninstalling networkx-2.4:
      Successfully uninstalled networkx-2.4


## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [14]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [16]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 1
}

Fix baseline parameters and train baseline model

In [17]:
CONTROL_NAME = 'xgb baseline'

baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [46]:
xgb_baseline_space = dict(**{**static_params, **baseline_params})
xgb_baseline_space

{'colsample_bytree': 0.9,
 'eta': 0.15,
 'eval_metric': 'logloss',
 'featuresCol': 'features',
 'gamma': 0.9,
 'labelCol': 'label',
 'max_depth': 6,
 'min_child_weight': 50.0,
 'nthread': 1,
 'num_round': 20,
 'nworkers': 1,
 'objective': 'binary:logistic',
 'predictionCol': 'prediction',
 'silent': 0,
 'subsample': 0.9}

In [18]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.7271829091740122

In [19]:
all_metrics = {}

In [20]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [21]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'max_depth': 6, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 50.0, 'nworkers': 1, 'silent': 0, 'eval_metric': 'logloss', 'subsample': 0.9, 'gamma': 0.9, 'eta': 0.5, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 10, 'labelCol': 'label'}
LOG-LOSS: 0.5078780628280688, ROC-AUC: 0.7287123616296299
SPACE:                                                                          
{'max_depth': 6, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 50.0, 'nworkers': 1, 'silent': 0, 'eval_metric': 'logloss', 'subsample': 0.9, 'gamma': 0.9, 'eta': 0.2, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 40, 'labelCol': 'label'}
LOG-LOSS: 0.5050163785773465, ROC-AUC: 0.7330245029524183                       
SPACE:                                                                          
{'max_depth': 6, 'objective'

LOG-LOSS: 0.5079063546012836, ROC-AUC: 0.7291571642723353                        
SPACE:                                                                           
{'max_depth': 6, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 50.0, 'nworkers': 1, 'silent': 0, 'eval_metric': 'logloss', 'subsample': 0.9, 'gamma': 0.9, 'eta': 0.3, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.500513959536055, ROC-AUC: 0.7386819178312383                         
SPACE:                                                                           
{'max_depth': 6, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 50.0, 'nworkers': 1, 'silent': 0, 'eval_metric': 'logloss', 'subsample': 0.9, 'gamma': 0.9, 'eta': 0.2, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 40, 'labelCol': 'label'}
LOG-LOSS: 0.5050163785773465, ROC-AUC: 0.73302450295242

In [23]:
best

{'eta': 0, 'num_round': 3}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [25]:
eta = eta_choice[best['eta']]  # change me!
num_round = num_round_choice[best['num_round']]  # change me!
eta, num_round

(0.5, 100)

### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

In [26]:
%%time

max_depth_choice = [4, 5, 6, 7, 8]
min_child_weight_choice = [1.0, 2.0, 3.0]
gamma_choice = [0.0, 0.2, 0.5, 0.75, 0.9]
subsample_choice = [0.8, 0.9, 1.0]
colsample_bytree_choice = [0.8, 0.9, 1.0]

space = {
    # Optimize
    'max_depth': hp.choice('max_depth', max_depth_choice),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),
    'gamma': hp.choice('gamma', gamma_choice),
    'subsample': hp.choice('subsample', subsample_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choice),
    
    # Fixed
    'eta': eta,
    'num_round': num_round,
    
    **static_params
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'max_depth': 4, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 2.0, 'nworkers': 1, 'silent': 0, 'eval_metric': 'logloss', 'subsample': 0.9, 'gamma': 0.2, 'eta': 0.5, 'colsample_bytree': 0.8, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.5019398551958152, ROC-AUC: 0.7368765374925562
SPACE:                                                                          
{'max_depth': 5, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 3.0, 'nworkers': 1, 'silent': 0, 'eval_metric': 'logloss', 'subsample': 0.8, 'gamma': 0.2, 'eta': 0.5, 'colsample_bytree': 0.8, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.5010071631844533, ROC-AUC: 0.7382098065757006                       
SPACE:                                                                           
{'max_depth': 8, 'objective

LOG-LOSS: 0.5021191715432518, ROC-AUC: 0.7367349808277762                        
SPACE:                                                                           
{'max_depth': 6, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 3.0, 'nworkers': 1, 'silent': 0, 'eval_metric': 'logloss', 'subsample': 1.0, 'gamma': 0.2, 'eta': 0.5, 'colsample_bytree': 1.0, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.5011761807150377, ROC-AUC: 0.7376583980064656                        
SPACE:                                                                           
{'max_depth': 4, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 3.0, 'nworkers': 1, 'silent': 0, 'eval_metric': 'logloss', 'subsample': 0.8, 'gamma': 0.75, 'eta': 0.5, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.502032681826571, ROC-AUC: 0.736831610509347

In [28]:
max_depth = max_depth_choice[best['max_depth']]
min_child_weight = min_child_weight_choice[best['min_child_weight']]
gamma = gamma_choice[best['gamma']]
subsample = subsample_choice[best['subsample']]
colsample_bytree = colsample_bytree_choice[best['colsample_bytree']]
max_depth, min_child_weight, gamma, subsample, colsample_bytree

(5, 1.0, 0.9, 0.8, 0.9)

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [29]:
%%time

lambda_choice = [0.0, 0.1, 0.2, 0.3, 0.5, 1.0, 2.0, 3.0]
alpha_choice = [0.0, 0.1, 0.2, 0.3, 0.5, 1.0, 2.0, 3.0]

space = {
    # Optimize
    'reg_lambda': hp.choice('lambda', lambda_choice),
    'alpha': hp.choice('alpha', alpha_choice),
    
    # Fixed
    'eta': eta,
    'num_round': num_round,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    
    **static_params
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'lambda': 0.1, 'max_depth': 5, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 1.0, 'eval_metric': 'logloss', 'nworkers': 1, 'silent': 0, 'alpha': 0.3, 'subsample': 0.8, 'gamma': 0.9, 'eta': 0.5, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.5013403083742151, ROC-AUC: 0.7376794654435684
SPACE:                                                                           
{'lambda': 2.0, 'max_depth': 5, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 1.0, 'eval_metric': 'logloss', 'nworkers': 1, 'silent': 0, 'alpha': 0.5, 'subsample': 0.8, 'gamma': 0.9, 'eta': 0.5, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.5008197045921008, ROC-AUC: 0.7382888938972683                        
SPACE:                                           

LOG-LOSS: 0.5009948183757488, ROC-AUC: 0.7382672871411703                         
SPACE:                                                                            
{'lambda': 0.0, 'max_depth': 5, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 1.0, 'eval_metric': 'logloss', 'nworkers': 1, 'silent': 0, 'alpha': 3.0, 'subsample': 0.8, 'gamma': 0.9, 'eta': 0.5, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.5005040925273453, ROC-AUC: 0.7387455526153412                         
SPACE:                                                                            
{'lambda': 0.2, 'max_depth': 5, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 1.0, 'eval_metric': 'logloss', 'nworkers': 1, 'silent': 0, 'alpha': 3.0, 'subsample': 0.8, 'gamma': 0.9, 'eta': 0.5, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'la

In [33]:
lambda_ = lambda_choice[best['lambda']]
alpha = alpha_choice[best['alpha']]
lambda_, alpha

(0.2, 3.0)

### Lower the learning rate and decide the optimal parameters

In [ ]:
%%time

space = {
    # Optimize
    'eta': hp.uniform('eta', eta / 10, eta * 2),
    
    # Fixed
    'num_round': num_round,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'alpha': alpha,
    'reg_lambda': lambda_,
    
    **static_params
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'lambda': 0.2, 'max_depth': 5, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 1.0, 'eval_metric': 'logloss', 'nworkers': 1, 'silent': 0, 'alpha': 3.0, 'subsample': 0.8, 'gamma': 0.9, 'eta': 0.2730904686686966, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.5020502685752237, ROC-AUC: 0.7368689493298782
SPACE:                                                                           
{'lambda': 0.2, 'max_depth': 5, 'objective': 'binary:logistic', 'nthread': 1, 'predictionCol': 'prediction', 'min_child_weight': 1.0, 'eval_metric': 'logloss', 'nworkers': 1, 'silent': 0, 'alpha': 3.0, 'subsample': 0.8, 'gamma': 0.9, 'eta': 0.05459785327990466, 'colsample_bytree': 0.9, 'featuresCol': 'features', 'num_round': 100, 'labelCol': 'label'}
LOG-LOSS: 0.5079548449715215, ROC-AUC: 0.729304051761509                         
SPACE:            

In [39]:
eta = best['eta']
eta

0.438962792765098

### Final

In [40]:
xgb_hyperopt_space = {
    'num_round': num_round,
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    'alpha': alpha,
    'reg_lambda': lambda_,
    'eta': eta,
    **static_params
}
xgb_hyperopt = XGBoostEstimator(**xgb_hyperopt_space).fit(train_df)

In [41]:
all_metrics['xgb_hyperopt'] = dict(
    logloss=logloss(xgb_hyperopt, test_df, probabilities_col='probabilities'),
    rocauc=rocauc(xgb_hyperopt, test_df, probabilities_col='probabilities'),
)

In [42]:
get_ate(all_metrics, CONTROL_NAME)

,metric,xgb_hyperopt ate %
0,logloss,-1.848411
1,rocauc,1.689775


---
## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

In [48]:
from pyspark.ml.classification import LogisticRegression
LogisticRegression?

In [53]:
logreg_baseline_space = dict(
    regParam=0.0,
    elasticNetParam=0.0,
)
lr_model = LogisticRegression(**logreg_baseline_space).fit(train_df)

In [54]:
all_metrics['logreg_baseline'] = dict(
    logloss=logloss(lr_model, test_df, probabilities_col='probability'),
    rocauc=rocauc(lr_model, test_df, probabilities_col='probability'),
)
get_ate(all_metrics, CONTROL_NAME)

,logreg_baseline ate %,metric,xgb_hyperopt ate %
0,3.634401,logloss,-1.848411
1,-3.313159,rocauc,1.689775


In [55]:
def objective(space):
    model = LogisticRegression(**space).fit(train_df)
    log_loss = logloss(model, val_df, probabilities_col='probability')
    roc_auc = rocauc(model, val_df, probabilities_col='probability')
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))
    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK}

In [57]:
%%time

space = {
    # Optimize
    'regParam': hp.uniform('regParam', 0.0, 1.0),
    'elasticNetParam': hp.uniform('elasticNetParam', 0.0, 1.0),
}

trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5            
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5576806937855293, ROC-AUC: 0.6862189228452311                       
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5599357096054804, ROC-AUC: 0.6728079787248457                       
 45%|████▌     | 9/20 [00:47<01:02,  5.64s/trial, best loss: 0.5576806937855293]

Exception ignored in: 
<object repr() failed>
Traceback (most recent call last):

  File "/home/stanislavv/Apps/Conda/envs/bd/lib/python3.5/site-packages/pyspark/ml/wrapper.py", line 40, in __del__

if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError
: 
'XGBoostEstimator' object has no attribute '_java_obj'


LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                       
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                       
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                       
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                       
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                       
LOG-LOSS: 0.5654206379632322, ROC-AUC: 0.679885760159641                         
LOG-LOSS: 0.5524166703299029, ROC-AUC: 0.6948106476914899                        
LOG-LOSS: 0.564929339011796, ROC-AUC: 0.6624021327332127                         
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                       
LOG-LOSS: 0.5744867160831884, ROC-AUC: 0.5                                       
100%|██████████| 20/20 [01:46<00:00,  5.34s/trial, best loss: 0.5524166703299029]
CPU times: user 1

In [60]:
logreg_hyperopt_space = best
logreg_hyperopt_space

{'elasticNetParam': 0.009628408454389503, 'regParam': 0.42429458052784585}

In [61]:
lr_model = LogisticRegression(**logreg_hyperopt_space).fit(train_df)
all_metrics['logreg_hyperopt'] = dict(
    logloss=logloss(lr_model, test_df, probabilities_col='probability'),
    rocauc=rocauc(lr_model, test_df, probabilities_col='probability'),
)
get_ate(all_metrics, CONTROL_NAME)

,logreg_baseline ate %,logreg_hyperopt ate %,metric,xgb_hyperopt ate %
0,3.634401,7.895084,logloss,-1.848411
1,-3.313159,-4.116621,rocauc,1.689775


---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте справнение на отложенной выборке

In [62]:
all_metrics

{'logreg_baseline': {'logloss': 0.533040613185152,
  'rocauc': 0.7000818520911786},
 'logreg_hyperopt': {'logloss': 0.5549553205913881,
  'rocauc': 0.694264210319586},
 'xgb baseline': {'logloss': 0.5143471764449784, 'rocauc': 0.7240714922149476},
 'xgb_hyperopt': {'logloss': 0.5048399273366083, 'rocauc': 0.7363066720756228}}

Итоговая таблица

In [63]:
get_ate(all_metrics, CONTROL_NAME)

,logreg_baseline ate %,logreg_hyperopt ate %,metric,xgb_hyperopt ate %
0,3.634401,7.895084,logloss,-1.848411
1,-3.313159,-4.116621,rocauc,1.689775


# Submission

In [91]:
KAGGLE_DATA_PATH = '/home/stanislavv/data/criteo/kaggle-data'

KAGGLE_TRAIN_PATH = os.path.join(KAGGLE_DATA_PATH, 'train.csv')
KAGGLE_TEST_PATH = os.path.join(KAGGLE_DATA_PATH, 'test.csv')
KAGGLE_SUBMISSION_PATH = os.path.join(KAGGLE_DATA_PATH, 'submission.csv')

In [92]:
train_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + KAGGLE_TRAIN_PATH)

train_df = train_df.fillna(0, subset=num_columns)
train_df = pipeline_model \
    .transform(train_df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

In [93]:
best_model = XGBoostEstimator(**xgb_hyperopt_space).fit(train_df)

In [94]:
test_df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + KAGGLE_TEST_PATH)

test_df = test_df.fillna(0, subset=num_columns)
test_df = pipeline_model.transform(test_df).select('id', 'features')

In [95]:
pred = best_model.transform(test_df).select('id', 'probabilities')
pred = pred.toPandas()
pred['proba'] = pred['probabilities'].map(lambda p: p[1] if not pd.isna(p[1]) else 0.)
pred.head()

,id,probabilities,proba
0,566935904713,"[0.6657132506370544, 0.33428674936294556]",0.334287
1,566935904715,"[0.7560364603996277, 0.24396353960037231]",0.243964
2,566935904727,"[0.6948901414871216, 0.3051098883152008]",0.305110
3,566935904737,"[0.5388923287391663, 0.46110767126083374]",0.461108
4,566935904741,"[0.7998224496841431, 0.20017752051353455]",0.200178


In [96]:
pred[['id', 'proba']].to_csv(KAGGLE_SUBMISSION_PATH, index=False)